In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template(
    "Hi, what would be the best copy for a {business} website?"
)
prompt = template.format(business="golf lounge")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)

"Welcome to the Ultimate Golf Lounge Experience!\n\nAt [Golf Lounge Name], we are passionate about creating a haven for golf enthusiasts like you. Step into our world-class facility and indulge in the perfect blend of golf, relaxation, and entertainment. Whether you're a seasoned golfer or just starting out, our golf lounge offers an unparalleled experience that will leave you wanting more.\n\nUnwind in our luxurious lounge area, designed to provide a comfortable and stylish atmosphere. Catch up with friends, fellow golfers, or business associates while enjoying a refreshing drink from our fully stocked bar. Our friendly and knowledgeable staff is always on hand to ensure your every need is met, making your visit truly unforgettable.\n\nTake your game to the next level with our cutting-edge golf simulators. Immerse yourself in a virtual golfing paradise, where you can play on world-renowned courses without leaving the comfort of our lounge. Our state-of-the-art technology accurately re

In [2]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

template = ChatPromptTemplate.from_messages([
    ("system", "You are a copywriter for digital products. You only reply in {language}."),
    ("ai", "Hi, I am your chatbot."),
    ("human", "Hi, what would be the best copy for a {business} website?")])


prompt = template.format_messages(
    language="Spanish",
    business="Coffe shop"
)

chat.predict_messages(prompt)

AIMessage(content='¡Bienvenidos al mejor lugar para disfrutar del café! En nuestra cafetería, encontrarás el equilibrio perfecto entre sabor, calidad y ambiente acogedor. Nuestros expertos baristas están listos para deleitarte con una amplia variedad de cafés de origen, cuidadosamente seleccionados y preparados con pasión. Además, ofrecemos deliciosos postres y bocadillos que complementarán tu experiencia. Ven y descubre por qué somos el destino favorito de los amantes del café. ¡Te esperamos con una taza de café perfecta!')

In [5]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.split(',')
        return list(map(str.strip, items))

p = CommaOutputParser()
p.parse("Hello, How, Are,You,?")

['Hello', 'How', 'Are', 'You', '?']

In [14]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. You should answer only in list of maximum {max}, separated with a comma. Do NOT reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max=10,
    question="What are the names of planets?"
)

message = chat.predict_messages(prompt).content

p = CommaOutputParser()
planets = p.parse(message)
print(planets)

# planets

['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']


In [19]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max": 20,
    "question": "What are the names of Pokemons?"
})

['Pikachu',
 'Bulbasaur',
 'Charizard',
 'Squirtle',
 'Jigglypuff',
 'Eevee',
 'Snorlax',
 'Mewtwo',
 'Gyarados',
 'Dragonite',
 'Blastoise',
 'Venusaur',
 'Lapras',
 'Arcanine',
 'Gengar',
 'Machamp',
 'Alakazam',
 'Mew',
 'Zapdos',
 'Articuno']

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a chef. You create recipes that are easy to follow for any type of cuisine with ingredients easy to find."),
    ("human", "I want to cook a {cuisine} food. Give me a simple recipe and ingredients.")
])

chef_chain = chef_prompt | chat 

veg_chef_promt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative recipes. You don't radically modify the recipe. If there is no alternative for a food or ingredient, just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_promt | chat 

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "Indian"
})
 

Sure! How about making a delicious and easy Indian dish called Chicken Tikka Masala? Here's a simple recipe and the ingredients you'll need:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 large onion, finely chopped
- 3 cloves of garlic, minced
- 1 tablespoon ginger, grated
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon turmeric powder
- 1 teaspoon paprika
- 1 teaspoon garam masala
- 1 cup tomato puree
- 1 cup heavy cream
- Salt, to taste
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 teaspoon of ground cumin, 1 teaspoon of ground coriander, turmeric powder, paprika, and salt. Mix well.
2. Add the chicken pieces to the marinade and coat them evenly. Let it marinate for at least 30 minutes, or refrigerate overnight for better flavor.
3. Heat the vegetable oil in a large pan over medium heat. Add the chopped

AIMessageChunk(content="Sure! Here's a vegetarian alternative recipe for Chicken Tikka Masala:\n\nIngredients:\n- 500g firm tofu, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt if desired)\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 large onion, finely chopped\n- 3 cloves of garlic, minced\n- 1 tablespoon ginger, grated\n- 2 teaspoons ground cumin\n- 2 teaspoons ground coriander\n- 1 teaspoon turmeric powder\n- 1 teaspoon paprika\n- 1 teaspoon garam masala\n- 1 cup tomato puree\n- 1 cup coconut milk (or heavy cream alternative)\n- Salt, to taste\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. In a bowl, combine the yogurt, lemon juice, 1 teaspoon of ground cumin, 1 teaspoon of ground coriander, turmeric powder, paprika, and salt. Mix well.\n2. Add the tofu pieces to the marinade and coat them evenly. Let it marinate for at least 30 minutes, or refrigerate overnight for better flavor.\n3. Heat the vegetable oil in a lar